In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet, download_model
from springstone.utils import bollinger_bands, moving_average, prophet_preprocessing, prophet_non_business_days
from springstone.utils import basic_recommendation, temp_data_predict, next_business_day
from springstone.trainer import Trainer
from springstone.params import MODEL_TYPE
from springstone.predict import predict_from_model
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_forecast_component_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib
from termcolor import colored
from datetime import date, timedelta
import holidays

In [3]:
ticker = 'AAPL'

In [4]:
hist = get_data(ticker, end='2022-03-04')

[*********************100%***********************]  1 of 1 completed


In [5]:
hist

,Open,High,Low,Close,Volume
Date,,,,,
2012-02-21,18.102858,18.387501,18.004286,18.387501,605595200
2012-02-22,18.324286,18.410357,18.181070,18.322857,483302400
2012-02-23,18.395714,18.493929,18.196428,18.442499,568027600
2012-02-24,18.559643,18.674999,18.522858,18.657499,415072000
2012-02-27,18.618214,18.875000,18.438572,18.777143,547582000
...,...,...,...,...,...
2022-02-25,163.839996,165.119995,160.869995,164.850006,91974200
2022-02-28,163.059998,165.419998,162.429993,165.119995,94869100
2022-03-01,164.699997,166.600006,161.970001,163.199997,83474400


In [ ]:
df_nbd = prophet_non_business_days(hist)

In [ ]:
# Using the Trainer class
df_train, df_test = create_train_test(hist)
trainer_prophet = Trainer(model="prophet",X=df_train, y=None, non_business_days=df_nbd)
df_test_prophet = create_df_for_prophet(df_test)
trainer_prophet.run()
trainer_prophet.save_model_locally(ticker,'prophet')

In [ ]:
# Without the Trainer class
df_prophet = prophet_preprocessing(hist, 'Close')
df_train, df_test = create_train_test(df_prophet)
m_prophet = Prophet(holidays=df_nbd)
m_prophet.fit(df_train)
joblib.dump(m_prophet, 'model_prophet_AAPL.joblib')

In [ ]:
df_test_prophet = create_df_for_prophet(df_test)
#y_pred = trainer_prophet.predict(df_test_prophet[:5][['ds']])
y_pred = predict_from_model(ticker, 'prophet', df_test_prophet)

In [ ]:
y_pred[0]

In [ ]:
basic_recommendation(ticker)

In [ ]:
next_business_day(date.today())

In [ ]:
n_days_prediction=1
future = pipeline.make_future_dataframe(periods=n_days_prediction)
future

In [ ]:
forecast = pipeline.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
m_prophet.plot(forecast);

In [ ]:
m_prophet.plot_components(forecast);

In [ ]:
#plot_forecast_component_plotly(m_prophet, forecast, 'non business day')

In [ ]:
forecast = forecast[- n_days_prediction:][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
forecast

In [ ]:
df_test[:n_days_prediction]

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
pbrint(f'MAPE: {mape_prophet}')

In [ ]:
def predict_from_model(ticker, model_type, X):
    """Predicts from a saved model for a ticker and a model_type
       Input:
            ticker: ticker used by the model
            model_type: type of model used such as prohet or RNN
            X: data
       Ouptut: return a dataframe with predicted values"""
    model = download_model(ticker, model_type)
    if model_type == "prophet":
        y_pred = model['prophet_model'].prophet.predict(X)['yhat']
    else:
        y_pred = model.predict(X)
        
    return y_pred

In [ ]:
cd notebooks

In [ ]:
download_model('AAPL', 'prophet')